In [215]:
import logging
import json
import requests
import faiss
import numpy as np
import torch

OLLAMA_BASE_URL = "http://localhost:11434" # local ollama url
MODEL_NAME = "llama3.2:latest" # model we are using to chat
EMBEDDING_MODEL_NAME = "bge-m3:latest" # model we are using to embed text

In [216]:
def generate_embeddings(document_list:list, model_name:str="bge-m3:latest") -> list[torch.FloatTensor]:
    embeddings = []
    for document in document_list:
        embedding_list = _generate_embeddings(document, model_name)
        embedding = torch.FloatTensor(embedding_list)
        embeddings.append(embedding)
    assert len(embeddings) == len(document_list)
    return torch.stack(embeddings, dim=0)

def _generate_embeddings(document:str, model_name:str="bge-m3:latest") -> list[float]:
    """Generate embeddings for the given text using the specified model."""
    try:
        logging.debug("Generating embeddings for {}".format(document))
        # Send a POST request to generate embeddings
        url = f"{OLLAMA_BASE_URL}/api/embeddings"
        data = {
            "prompt": document,
            "model": model_name
        }
        response = requests.post(url, json=data)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            embeddings = response.json()["embedding"]
            return embeddings
        else:
            logging.error(f"Failed to generate embeddings. Status code: {response.status_code}")
            logging.error("Response:", response.text)
            return []
    
    except requests.ConnectionError:
        logging.error("Failed to connect to the Ollama server. Make sure it is running locally and the URL is correct.")
        return []
    except json.JSONDecodeError:
        logging.error("Failed to parse JSON response from Ollama server.")
        return []
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return []


In [217]:
documents = [
    "Python is a programming language that lets you work quickly.",
    "Machine learning is a method of data analysis that automates analytical model building.",
    "Artificial Intelligence (AI) is intelligence demonstrated by machines.",
    "Natural language processing (NLP) is a field of AI that focuses on the interaction between computers and human language.",
]

# Generate embeddings for documents
document_embeddings = generate_embeddings(documents)
logging.info(f"Generated Document Embeddings: {document_embeddings.shape}")

# Build a FAISS index for fast similarity search
index = faiss.IndexFlatL2(document_embeddings.shape[1])


def retrieve_documents(query, top_k=2):
    # Generate query embedding
    query_embedding = generate_embeddings([query])
    # Search for the most similar documents
    _, indices = index.search(np.array(query_embedding), top_k)
    # Return the retrieved documents
    print(f"Found {indices[0]} documents")
    return [documents[i] for i in indices[0]]


# Example usage
query = "Tell me about artificial intelligence"
retrieved_docs = retrieve_documents(query)
print("Retrieved Documents: {}".format(retrieved_docs))


Found [-1 -1] documents
Retrieved Documents: ['Natural language processing (NLP) is a field of AI that focuses on the interaction between computers and human language.', 'Natural language processing (NLP) is a field of AI that focuses on the interaction between computers and human language.']


In [218]:
logging.debug("Retrieved Documents: {}".format(retrieved_docs))
for doc in retrieved_docs:
    logging.info(doc)

In [219]:
def to_curl(json_data, base_url):
    curl_command = "curl -X POST -H \"Content-Type: application/json\" -d '" + json.dumps(json_data) + "' " + base_url + "/api/chat"
    return curl_command


In [220]:

def chat_with_ollama(prompt, model_name ="llama3.2:latest"):
    try:
        url = f"{OLLAMA_BASE_URL}/api/chat"
        data = {
            "model": model_name,
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream" : False
        }
        response = requests.post(url, json=data)
        if response.status_code == 200:
            return response.json()["message"]["content"]
        else:
            logging.error(f"Failed to generate response. Status code: {response.status_code}")
            logging.error("Response:", response.text)
            return None
    
    except requests.ConnectionError:
        logging.error("Failed to connect to the Ollama server. Make sure it is running locally and the URL is correct.")
        return None
    except json.JSONDecodeError:
        logging.error("Failed to parse JSON response from Ollama server.")
        return None
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        return None


In [221]:
def format_docs(docs):
    formatted_docs = ""
    for i, doc in enumerate(docs):
        formatted_docs += f"{i+1}. {doc}\n"
    return formatted_docs

def generate_answer(query, retrieved_docs):
    # Combine the query with the retrieved documents to form the context
    context = f"Using this information :\n { format_docs(retrieved_docs) }\n Can you answer this question: {query}."
    print(f"Generated Context:\n {context}")

    # Return the generated answer
    return chat_with_ollama(context)


In [222]:
def rag_system(query):
    # Retrieve relevant documents based on the query
    retrieved_docs = retrieve_documents(query)
    # Generate a response using the retrieved documents as context
    generated_answer = generate_answer(query, retrieved_docs)
    
    return generated_answer


In [223]:
query = "What is Natural Language Processing?"

result = rag_system(query)
print(result)

Found [-1 -1] documents
Generated Context:
 Using this information :
 1. Natural language processing (NLP) is a field of AI that focuses on the interaction between computers and human language.
2. Natural language processing (NLP) is a field of AI that focuses on the interaction between computers and human language.

 Can you answer this question: What is Natural Language Processing?.
A nice example of repetition!

According to the given information, Natural Language Processing (NLP) is a field of Artificial Intelligence (AI) that focuses on the interaction between computers and human language.
